## 라이브러리 사용

In [1]:
# 사용할 라이브러리
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
import joblib

# 경고창 무시
import warnings
warnings.filterwarnings(action='ignore')

## 모듈화

긍정 사전 모듈화

In [2]:
def pos_dict():
    positive = pd.read_csv('ai/data_list/positive_dictionary.csv', encoding='utf-8')
    positive_dict = positive['word'].values
    positive_dict.tolist()
    positive_word = ['빠르고', '굿', '더', '좋아', '정말', '대박', '빨리', '예뻐요',
                     '예쁘', '실용', '딱', '귀엽', '넘', '저렴', '좋고', '좋다', '좋아',
                     '편한', '편함', '편하다', '재구매', '구매', '가성비', '다만', '젤',
                     '빠른', '배송', '빠', '여유', '마음', '곱네요', '이쁨', '최고', '추가',
                     '좋은', '감사', '신용', '귀엽고', '생각보다', '저렴한', '모두', '맘에', 
                     '들다', '맘에는', '좋아요', 'ㅎㅎ']
    positive_dict = np.append(positive_dict, positive_word)

    return positive_dict

In [3]:
joblib.dump(pos_dict(), 'pos_dict.pkl')

['pos_dict.pkl']

부정 사전 모듈화

In [4]:
def neg_dict():
    negative = pd.read_csv('ai/data_list/negative_dictionary.csv', encoding='utf-8')
    negative_dict = negative['word'].values
    negative_dict.tolist()
    negative_word = ['엉망', '없이', '가고', '조금', '비추', '불편', '별로',
                     '안', '때', '작은', '빨', '비싼', '별루',
                     '몰랐', '이상', '오랫', '않아요', '불친절', '불쾌', '최악', '떨어짐',
                     '빠지', '없', '높아', '근데', '따로', '번졌', '않',
                     '깔끄', '의심', '폭망', '떡', '걱정']
    negative_dict = np.append(negative_dict, negative_word)
    
    return negative_dict

In [5]:
joblib.dump(neg_dict(), 'neg_dict.pkl')

['neg_dict.pkl']

In [6]:
pos_dict = joblib.load('pos_dict.pkl')

In [7]:
neg_dict = joblib.load('neg_dict.pkl')

In [8]:
def plus_label():
    review = pd.read_table('ai/data_list/ratings_total.txt', names=['review_score', 'review_content'])
    review = review[:8000]
   
    review['review_content'] = review['review_content'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ가-힣]+', '', x))
    review.drop_duplicates(subset=['review_content'], inplace=True)

    tfidf1 = TfidfVectorizer()
    tfidf2 = TfidfVectorizer()

    tfidf1.fit_transform(pos_dict).toarray()
    tfidf2.fit_transform(neg_dict).toarray()
    
    review['label'] = np.nan
    count = len(review)
    for i in range(count):
        positive_words_count=tfidf1.transform([review['review_content'][i]]).toarray().sum()
        negative_words_count=tfidf2.transform([review['review_content'][i]]).toarray().sum()
        if positive_words_count - negative_words_count > 0:
            review.at[i, 'label'] = 1
        else:
            review.at[i, 'label'] = 0
    return review

In [9]:
joblib.dump(plus_label(), 'plus_label.pkl')

['plus_label.pkl']

In [10]:
plus_label = joblib.load('plus_label.pkl')

In [11]:
def tfidf():    
    review = plus_label
    
    X_train, X_test, y_train, y_test = train_test_split(
            review['review_content'],
            review['label'],
            test_size=0.2,
            random_state=42
    )
    
    stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
    stopwords = sum(stopwords, [])
    
    tfidf = TfidfVectorizer(lowercase = True, stop_words=stopwords)
    tfidf.fit(X_train)
    
    X_train =  tfidf.transform(X_train)
    X_test= tfidf.transform(X_test)
    
    return tfidf, X_train, y_train, X_test

In [12]:
joblib.dump(tfidf(), 'tfidf.pkl')

['tfidf.pkl']

In [13]:
tfidf = joblib.load('tfidf.pkl')

In [14]:
def SA_lr():
    tfidf_list = tfidf
    SA_lr = LogisticRegression(random_state = 0)
    SA_lr.fit(tfidf_list[1], tfidf_list[2])
    return SA_lr

In [15]:
joblib.dump(SA_lr(), 'SA_lr.pkl')

['SA_lr.pkl']

In [16]:
SA_lr = joblib.load('SA_lr.pkl')

In [17]:
## 하나만 돌릴 때

In [35]:
def labeling(df, column, i):
    tfidf_list = tfidf
    tfidf_module = tfidf_list[0]
    SA_module = SA_lr
    data = df[column][i]
    data = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", data)
    pn_tfidf = tfidf_module.transform([data])
    predict = SA_module.predict(pn_tfidf)
    if(predict[0] == 1):
        df['review_label'] = df['review_label'].astype(str)
        df.at[i, 'review_label'] = '1F60A'
    else:
        df['review_label'] = df['review_label'].astype(str)
        df.at[i, 'review_label'] = '1F621'
    print(df['review_label'][i])

In [36]:
labeling(review2, 'review_content', 11)

1F60A


In [37]:
review2

,review_idx,product_idx,user_id,review_content,review_img,review_score,review_rgstdate,review_label
0,1,1,admin,색감이 곱네요,good!.png,4.5,2022-05-13 15:10:51,1F621
1,3,1,aaaaaa1234,색이 오묘한게 정말 예뻐요 여름에 진짜 잘 입을 것 같아용 케키케,good!.png,5.0,2022-05-13 15:20:53,1F60A
2,4,1,use06048,우선 옷은 되게 얇아요! 그리고 제가 키가 큰 편이라 너무 짧고 작을까봐 걱정했는데...,good!.png,4.5,2022-05-13 15:20:53,1F621
3,5,1,qwer,색감이 너무 마음에 들어요! 사이즈도 괜찮고 디자인도 예뻐요~,good!.png,4.0,2022-05-13 15:20:53,1F60A
4,6,1,sdf,받고 나서 깜짝 놀랐네요 벽돌색이라고 하고 파세요 상품 사진에 나온 것보다 한참 어...,BAD.png,0.5,2022-05-13 15:20:53,1F621
5,7,1,use06048,예뻐요 짱짱 그냥 고민 없이 사세요 늦게 산 걸 후회하는 중..,well...png,3.5,2022-05-13 15:20:53,1F621
6,8,1,aaaaaa1234,배송은 빠르고 이런 스타일 좋아하는 사람은 좋아하겠고 레이어드 해서 입힐 생각,well...png,4.0,2022-05-13 15:20:53,1F60A
7,9,1,qwer,생각보다 더 얇고 작아요 ㅎㅎ 편하게 입기 좋을거 같습니다,good!.png,3.0,2022-05-13 15:20:53,1F60A
8,10,1,use06048,키 골격 큰편이어서 작을까봐 걱정했는데 생각보다 괜찮네영 여름에 롱스커트랑 닥마에 ...,well...png,4.5,2022-05-13 15:20:53,1F60A
9,11,1,sdf,옷만 보면 이쁨 바지도 청이나 어두운 계열 잘 어울릴 거 같아여,good!.png,3.5,2022-05-13 15:20:53,1F621
